In [1]:
# Librairies
import pandas as pd
import re
import nltk
import itertools
from collections import Counter
import os
import matplotlib.pyplot as plt
from datetime import date

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Majikthise\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data import

Your data should be downloaded from https://apps.webofknowledge.com search engine. The format should be a .xls file.

Copy it in the same folder of the notebook and input the name in the code below.


In [4]:
# THIS IS THE ONLY CELL YOU SHOULD HAVE TO EDIT. For now.

username = 'your_username'
trialnumber = '000'  # let be safe use 3 numbers starting at 000.
exportnumber = '000'  # let be safe use 3 numbers starting at 000.


current = './data_examples/ILRA_EXAMPLE_Data_rjs_cs_cc.xls'  # your new dataset
compare_data = ['./data_examples/ILRA_EXAMPLE_Data_rotary_jet_spinning.xls']  # your previus datasets

In [5]:
# Ignore: testing data for the duplicates and select new entries algorithms.
#current = './data_examples/ILRA_EXAMPLE_Data_rotary_jet_spinning_short.xls'  # your new dataset
#compare_data = ['./data_examples/ILRA_EXAMPLE_Data_rotary_jet_spinning_end.xls', './data_examples/ILRA_EXAMPLE_Data_rotary_jet_spinning_long.xls']  # your previus datasets

exportdate = date.today().strftime("%Y%m%d")  # get the date, ISO format
file_export_ID = username+'_'+exportdate+'_'+exportnumber+'_'+trialnumber

current_data = pd.read_excel(current)  # import current xls in dataframe

compare_list = []
raw_data = pd.DataFrame()

if not compare_data:
    print("No previous search")
    raw_data = current_data.copy(deep=True)
    print(len(raw_data))
else:
    print ("Comparing files and removing duplicate from previous searches")
    for filename in compare_data:
        compare_list.append(pd.read_excel(filename))
        
    # recursively remove previous data and keep only new entries.
    compare_df = pd.concat(compare_list).drop_duplicates(keep='first').copy(deep=True)  # compare all and remove duplicates (keeping first apparition)
    raw_data = pd.concat([current_data, compare_df]).drop_duplicates(keep='first').copy(deep=True)  # compare with current data to keep all differences
    raw_data = pd.concat([compare_df, raw_data]).drop_duplicates(keep=False).copy(deep=True)  # compare with previous data to keep only the new data (remove all duplicates)
    print('Number of new entries:', len(raw_data))


print('Exporting ID for files:', file_export_ID)
# raw_data.head  # you can uncomment this line to check that the file appears correctly

No previous search
275
Exporting ID for files: your_username_20210928_000_000


# Data cleaning

1. duplicate removal
2. remove empty columns (NaN)
3. text cleaning

In [ ]:
# locate duplicates
raw_data['dup'] = raw_data.duplicated(subset=None, keep='first')
# Counting the number of duplicates
raw_data['dup'].value_counts()
# Creating a new dataframe without the duplicates
raw_data_noDup = raw_data[raw_data['dup'] == False]
# Deleting the column with the True and False because because it is no more useful
del raw_data_noDup['dup']

In [ ]:
# Removing useless columns (All articles have written nothing in those fields)
raw_data_useField = raw_data_noDup.dropna(axis=1, how='all')

In [ ]:
# Changing the type of NaN to string
# (For text cleaning everything need to be string)
raw_data_str = raw_data_useField.fillna("NaN")

In [ ]:
# Text cleaning (removal of "meaningless" words)
"""
SmartStoplist.txt
by Lisa Andreevna
Lisanka93/text_analysis_python_101. (n.d.). GitHub. Retrieved May 3, 2021,
from https://github.com/lisanka93/text_analysis_python_101
**** Note : nan is added to Lisa Andreevna's list ****
"""
# Definition of constant and variable
stop_words_file = 'SmartStoplist.txt'
stop_words = []
# Creating a list of stop words while reading the stop words's file
with open(stop_words_file, "r") as f:
    for line in f:
        stop_words.extend(line.split())
# Do not understand yet
stop_words = stop_words
"""
Definition of a cleaning function (preprocess before words analysis)
This function get a text and return a text (string) of stemmed word in
lowercase without stop words and any caracter except letter
"""
def preprocess(raw_text):
    """
    Keep only letters in the text (lowercase and capitals) using Regex (re).
    Replace all symboles with a blank space.
    """
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)
    # Change the capitals for lowercase AND split into a list of words (no expression)
    words = letters_only_text.lower().split()
    # Define a variable to receive only the useful crop (or not) words
    cleaned_words = []
    # Remove stop words (Take word in list of words and make a list of clean words)
    for word in words:
        if word not in stop_words:
            cleaned_words.append(word)
    # Stem word (Creating a new list of stemmed word with the clean one)
    stemmed_words = []
    for word in cleaned_words:
        word = nltk.stem.PorterStemmer().stem(word)
        stemmed_words.append(word)
    # After all those changes, convert back the final list into string
    return " ".join(stemmed_words)


In [ ]:
# Clean abstracts of all the articles of the research (overwrite)
raw_data_str['Abstract'] = raw_data_str['Abstract'].apply(preprocess)
raw_data_str['Article Title'] = raw_data_str['Article Title'].apply(preprocess)
raw_data_str['Author Keywords'] = raw_data_str['Author Keywords'].apply(preprocess)
clean_data = raw_data_str.copy()

# Data exploration

The following cell is function which will extract the number of occurences and the series of bigrams and trigrams in the data.

The defined function is then applied to the data, and return the analysis.

In [ ]:
# word count

def word_count(data):
    # input: data is 1 dataframe column containing the text for each entry in the dataset
    # Occurence of all the words
    # convert to a dataframe
    words_occ = pd.DataFrame(Counter(" ".join(data).split()).most_common(), columns=['word', 'count'])

    # Most common bigrams and trigrams in clean data
    # Puting all abstracts into a list
    data_list = data.tolist()
    # Defining variables
    all_bigrams = []
    all_trigrams = []

    # Creating list of bigrams and trigrams by abstracts, i.e. list[0]=allBigramOfAbs1
    for item in data_list:
        item_token = nltk.word_tokenize(item)
        all_bigrams.append(list(nltk.bigrams(item_token)))
        all_trigrams.append(list(nltk.trigrams(item_token)))

    # count occurences and ranking grams
    all_bigrams = list(itertools.chain.from_iterable(all_bigrams))  # reduce list of list to one list
    bigrams_occ = Counter(all_bigrams).most_common()
    
    all_trigrams = list(itertools.chain.from_iterable(all_trigrams))  # reduce list of list to one list
    trigrams_occ = Counter(all_trigrams).most_common()
    
    # Obtaining the most commons ones by abstracts for all of them
    '''
    top3_bi = []
    for bi_by_abst in all_bigrams:
        top3_bi_by_abst = Counter(bi_by_abst).most_common(3)
        top3_bi.append(top3_bi_by_abst)
    top3_tri = []
    for tri_by_abst in all_trigrams:
        top3_tri_by_abst = Counter(tri_by_abst).most_common(3)
        top3_tri.append(top3_tri_by_abst)
    #print(top3_tri)
    #abstract_grams_df = pd.DataFrame([list_bigrams, list_trigrams])
    '''
    return words_occ, bigrams_occ, trigrams_occ

#rval_abstracts = word_count(clean_data['Abstract'])
#rval_abstracts[2]

In [ ]:

#print(Counter(list(itertools.chain.from_iterable(rval_abstracts[1][0:1]))).most_common())
#a = (Counter(list(itertools.chain.from_iterable(rval_abstracts[1])))


# Visualization

This function will plot a graph of the top 10 words in the data category we analysed.

In [ ]:
def barchart_top10(rval, plottitle):
    fig, ax = plt.subplots()

    ax.barh(rval[0]['word'][0:50], rval[0]['count'][0:50])
    ax.invert_yaxis()  # labels read top-to-bottom
    ax.set_xlabel('Count')
    ax.set_title('Word count - top 10 in ' + plottitle)
    
    ### SAVE A BIG GRAPH
    # bigram, trigrams, grams

## Abstracts

In [ ]:
rval_abstracts = word_count(clean_data['Abstract'])
barchart_top10(rval_abstracts, 'Abstracts')
# uncomment to get a look
#rval_abstract[0][0:10]  # top 10

## Analysis in the titles

In [ ]:
rval_titles = word_count(clean_data['Article Title'])
barchart_top10(rval_titles, 'Article Title')

## Analysis in the Author's keywords

In [ ]:
rval_keywords = word_count(clean_data['Author Keywords'])
barchart_top10(rval_keywords, 'Author Keywords')

# Data save and export

In [ ]:
def save_data(rval, datatitle):

    # input: rval is the data extracted, datatitle is a string to identify the data saved
    # output: export data analysis to csv.
    
    # If there is no folder for the result create one
    os.makedirs('Results/' + file_export_ID, exist_ok=True)

    # Word count data
    rval[0].to_csv('./Results/' + file_export_ID + '/ILRA_WordsOccurence_' + file_export_ID +'_'+ datatitle + '.csv', sep=',')

    # bigrams
    bigrams_df = pd.DataFrame(list(rval[1]))
    bigrams_df.to_csv('./Results/' + file_export_ID + '/ILRA_bigrams_' + file_export_ID +'_'+ datatitle + '.csv', sep=',')
    #trigrams
    trigrams_df = pd.DataFrame(list(rval[2]))
    trigrams_df.to_csv('./Results/' + file_export_ID + '/ILRA_trigrams_' + file_export_ID +'_'+ datatitle + '.csv', sep=',')

In [ ]:
save_data(rval_abstracts, 'abstract')
save_data(rval_titles, 'title')
save_data(rval_keywords, 'keywords')

In [ ]:
rval_abstracts[1][0:10]  # sanity check of the top ten